This notebook is part of the `kikuchipy` documentation https://kikuchipy.org.
Links to the documentation won't work from the notebook.

# Virtual backscatter electron imaging

In [22]:
# exchange inline for qt5 for interactive plotting from the pyqt package
%matplotlib qt5

import os
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
import kikuchipy as kp


datadir = "../kikuchipy/data/"  # For saving data

## Interactive plotting

Angle resolved backscatter electron (BSE) imaging can be performed interactively
with the method
[plot_virtual_bse_intensity()](reference.rst#kikuchipy.signals.EBSD.plot_virtual_bse_intensity),
adopted from [pyxem](https://github.com/pyxem/pyxem), by integrating the
intensities within a part, e.g. a (10 x 10) pixel rectangular region of interest
(ROI), of the stack of EBSD patterns:

In [12]:
roi = hs.roi.RectangularROI(left=0, top=0, right=10, bottom=10)
roi

RectangularROI(left=0, top=0, right=10, bottom=10)

In [10]:
s.plot_virtual_bse_intensity(roi)

In [11]:
roi

RectangularROI(left=13, top=26, right=14, bottom=27)

IMAGE

Note that the position of the ROI on the detector is updated during the
interactive plotting. See
[HyperSpy's ROI user guide](http://hyperspy.org/hyperspy-doc/current/user_guide/interactive_operations_ROIs.html#region-of-interest-roi)
for more detailed use of ROIs.

The virtual image, created from integrating the intensities within the ROI, can
then be written to an image file using
[get_virtual_bse_intensity](reference.rst#kikuchipy.signals.EBSD.get_virtual_bse_intensity).

In [13]:
vbse = s.get_virtual_bse_intensity(roi)
vbse

<VirtualBSEImage, title: Virtual backscatter electron image, dimensions: (|75, 55)>

In [14]:
tempdir = datadir + "temp/"; os.mkdir(tempdir)
plt.imsave(tempdir + "vbse1.png", arr=vbse.data)

A [VirtualBSEImage](reference.rst#kikuchipy.signals.VirtualBSEImage) instance is
returned.

## Generate many virtual images

Sometimes we want to get many images from parts of the detector, e.g. like what
is demonstrated in the
[xcdskd project](https://xcdskd.readthedocs.io/en/latest/bse_imaging.html) with
the angle resolved virtual backscatter electron array (arbse/vbse array).
Instead of keeping track of multiple
[hyperspy.roi.BaseInteractiveROI](http://hyperspy.org/hyperspy-doc/current/api/hyperspy.roi.html#hyperspy.roi.BaseInteractiveROI)
objects, we can create a detector grid of a certain shape, e.g. (5, 5), and
obtain gray scale images, or combine multiple grid tiles in red, green and
channels to obtain RGB images.

First, we initialize a virtual BSE image generator,
[kikuchipy.generators.VirtualBSEGenerator](reference.rst#kikuchipy.generators.VirtualBSEGenerator),
with an [EBSD](reference.rst#kikuchipy.signals.EBSD) signal, in this case the
raw EBSD patterns without any background correction or other processing.

In [23]:
s = kp.data.nickel_ebsd_large(allow_download=True)
s

<EBSD, title: patterns Scan 1, dimensions: (75, 55|60, 60)>

In [24]:
vbse_gen = kp.generators.VirtualBSEGenerator(s)
vbse_gen

VirtualBSEGenerator for <EBSD, title: patterns Scan 1, dimensions: (75, 55|60, 60)>

We can set and plot the detector grid on one of the EBSD patterns, also coloring
one or more of the grid tiles red, green and blue, as is done in
<cite data-cite="nolze2017electron">Nolze et al. (2017)</cite>, by calling
[VirtualBSEGenerator.plot_grid](reference.rst#kikuchipy.generators.VirtualBSEGenerator.plot_grid).

In [25]:
vbse_gen.grid_shape

(5, 5)

In [27]:
vbse_gen.grid_shape = (10, 10)
red = [(7, 1), (8, 1), (8, 2), (9, 1), (9, 2)]
green = [(8, 4), (8, 5), (9, 4), (9, 5)]
blue = [(7, 8), (8, 7), (8, 8), (9, 7), (9, 8)]
p = vbse_gen.plot_grid(
    rgb_channels=[red, green, blue],
    visible_indices=True,  # Default
    pattern_idx=(10, 20),  # Default is (0, 0)
)
p

<EBSD, title: patterns Scan 1, dimensions: (|60, 60)>


As shown above, whether to show the grid tile indices or not is controlled with
the ``visible_indices`` argument, and which signal pattern to superimpose the
grid upon is controlled with the ``pattern_idx`` parameter.

.. figure:: _static/image/virtual_backscatter_electron_imaging/plot_grid.jpg
    :align: center
    :width: 450

    Detector grid tiles, with tiles to be used for creating an RGB image colored
    red, green and blue.

To obtain an RGB image from the detector grid tiles shown above, we use
:meth:`~kikuchipy.generators.VirtualBSEGenerator.get_rgb_image` (see the
docstring for all available parameters):

.. code-block::

    >>> vbse_rgb_img = vbse_gen.get_rgb_image(r=red, g=green, b=blue)
    >>> vbse_rgb_img
    <VirtualBSEImage, title: , dimensions: (|200, 149)>
    >>> vbse_rgb_img.plot()

.. figure:: _static/image/virtual_backscatter_electron_imaging/rgb_image.jpg
    :align: center
    :width: 450

    An RGB image formed from coloring three grey scale virtual BSE images red,
    green and blue.

To obtain one grey scale virtual BSE image from each grid tile, we use
:meth:`~kikuchipy.generators.VirtualBSEGenerator.get_images_from_grid`:

.. code-block::

    >>> vbse_imgs = vbse_gen.get_images_from_grid()
    >>> vbse_imgs
    <VirtualBSEImage, title: , dimensions: (5, 5|200, 149)>
    >>> vbse_imgs.plot()

.. figure:: _static/image/virtual_backscatter_electron_imaging/images.jpg
    :align: center
    :width: 100%

    25 grey scale virtual BSE images, one from each tile in a (5, 5) detector
    grid.

It might be desirable to normalize or scale the intensities in the images, as
shown e.g. in Fig. 9 in [Wright2015b]_. This can be done with
:meth:`~kikuchipy.signals.VirtualBSEImage.rescale_intensity` or
:meth:`~kikuchipy.signals.VirtualBSEImage.normalize_intensity`:

.. [Wright2015b] S. I. Wright, M. M. Nowell, R. De Kloe, P. Camus, T. Rampton, \
    "Electron imaging with an EBSD detector," *Ultramicroscopy* **148** (2015),
    doi: http://dx.doi.org/10.1016/j.ultramic.2014.10.002.

.. code-block::

    >>> vbse_imgs.data.dtype
    dtype('float32')
    >>> print(vbse_imgs.data.min(), vbse_imgs.data.max())
    5629.0 31810.0
    >>> vbse_imgs.rescale_intensity()
    Rescaling the image intensities:
    [########################################] | 100% Completed |  0.3s
    >>> print(vbse_imgs.data.min(), vbse_imgs.data.max())
    -1.0 1.0

To obtain a rectangular ROI from the grid, we can use
:meth:`~kikuchipy.generators.VirtualBSEGenerator.roi_from_grid`:

.. code-block::

    >>> roi = vbse_gen.roi_from_grid((3, 3))  # (Row, column)
    >>> roi
    RectangularROI(left=18, top=18, right=24, bottom=24)

In [ ]:
from shutil import rmtree
rmtree(tempdir)  # Remove files written to disk in this user guide